In [5]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 18})


### Явная схема

In [9]:
# функция, которая выдает итоговый двумерный массив решений в зависимости от dt
def get_T(dt):
    dx=0.1 
    global x_steps,t_steps,x,t
    x_steps=int(10/dx)
    t_steps=int(1/dt)

    
    x=np.linspace(0,10,x_steps)
    t=np.linspace(0,1,t_steps)
    T=np.zeros((t_steps,x_steps))
    
    # начальные и граничные условия
    T[0]=(2+12+6)/15*(x-5)**2*np.exp(-(x-5)**2)
    T[:0]=T[:,-1]=0
    a=dt/dx**2
    
    # цикл заполнения значений
    for i in range (t_steps-1):
        for j in range(1,x_steps-1):
            T[i+1][j]=T[i][j]+a*(T[i][j+1]-2*T[i][j]+T[i][j-1])
            
    return T

# функция,выдающая диффузию 
k=np.linspace(0,0.49,1000)
def get_gamma(dt):
    l=1-4*a*np.sin(np.pi*1/2*k)**2
    gamma=-np.log(np.real(l))/dt
    return gamma

In [10]:
# используем функции и строим графики


T1=get_T(0.005)
gamma1=get_gamma(0.005)

plt.figure(figsize=(15,8))
plt.title('Явная схема при dx=0.01 и dt=0.005')

plt.xlabel('X')
plt.ylabel('T')
time=np.array([0,0.1,0.2,0.3,0.5,1])
T1=np.append(T1,[T1[-1]],axis=0)
for i in time:
    plt.plot(x,T1[int(i*t_steps)],'-',label='t = '+str(i))
    plt.legend()

    
T2=get_T(0.01)
gamma2=get_gamma(0.01)

plt.figure(figsize=(15,8))
plt.title('Явная схема при dx=0.01 и dt=0.01')

plt.xlabel('X')
plt.ylabel('T')
plt.yscale('log')
time=np.array([0,0.1,0.2,0.3,0.5,1])
T2=np.append(T2,[T2[-1]],axis=0)
for i in time:
    plt.plot(x,T2[int(i*t_steps)],'-',label='t = '+str(i))
    plt.legend()

NameError: name 'a' is not defined

In [11]:
# строим диффузию

plt.figure(figsize=(15,8))
plt.plot(k,gamma1,label='dt/$dx^2$ = 0.5')
plt.plot(k,gamma2,label='dt/$dx^2$ = 1 ')
plt.xlabel('$ k $')
plt.ylabel('$\delta$')
plt.plot(k,(np.pi/dx*k)**2,label='исходное уравнение')
plt.legend()
plt.title("Диффузия")

NameError: name 'gamma1' is not defined

<Figure size 1080x576 with 0 Axes>

## Схема Кранка-Николсона

In [12]:
# функция прогонки
def tridiagonal(y):
    a_m=a/2
    b_m=1+2*a_m
    c_m=a_m
    d=np.zeros(x_steps)
    s=np.zeros(x_steps)
    for j in range(1,x_steps-1):
        f_m=-y[j]-a/2*(y[j-1]-2*y[j]+y[j+1])
        d[j+1]=c_m/(b_m-a_m*d[j])
        s[j+1]=(f_m-a_m*s[j])/(a_m*d[j]-b_m)
    #y[x_steps-1]=0
    for j in range(x_steps-1,0,-1):
        y[j-1]=y[j]*d[j]+s[j]
    return y

# функция решения
def Krank_Nikolson(dt):
    dx=0.1 
    global x_steps,t_steps,x,t
    x_steps=int(10/dx)
    t_steps=int(1/dt)

    
    x=np.linspace(0,10,x_steps)
    t=np.linspace(0,1,t_steps)
    T=np.zeros((t_steps,x_steps))
    
    # начальные и граничные условия
    T[0]=(2+12+6)/15*(x-5)**2*np.exp(-(x-5)**2)
    T[:0]=T[:,-1]=0
    a=dt/dx**2
    # цикл заполнения 
    for i in range(t_steps-1):
        T[i+1]=tridiagonal(T[i])
    return T

k=np.linspace(0,0.98,1000)
def get_gamma2(dt):
    l=(1-2*a*np.sin(np.pi/2*k)**2)/(1+2*a*np.sin(np.pi/2*k)**2)
    gamma=-np.log(np.abs(l))/dt
    return gamma

In [13]:
T1=Krank_Nikolson(0.005)
gamma1=get_gamma2(0.005)
plt.figure(figsize=(15,8))
plt.title('Cхема Кранка-Николсона при dx=0.01 и dt=0.005')
plt.xlabel('X,м')
plt.ylabel('T, К')
time=np.array([0,0.1,0.2,0.3,0.5,1])
T1=np.append(T1,[T1[-1]],axis=0)
for i in time:
    plt.plot(x,T1[int(i*t_steps)],'-',label='t = '+str(i))
    plt.legend()



T2=Krank_Nikolson(0.01)
gamma2=get_gamma2(0.01)

plt.figure(figsize=(15,8))
plt.title('Cхема Кранка-Николсона при dx=0.01 и dt=0.01')
plt.xlabel('X,м')
plt.ylabel('T,K')
time=np.array([0,0.1,0.2,0.3,0.5,1])
T2=np.append(T2,[T2[-1]],axis=0)
for i in time:
    plt.plot(x,T2[int(i*t_steps)],'-',label='t = '+str(i))
    plt.legend()


NameError: name 'a' is not defined

In [14]:
# строим диффузию

plt.figure(figsize=(15,8))
plt.plot(k,gamma1,label='dt/$dx^2$ = 0.5')
plt.plot(k,gamma2,label='dt/$dx^2$ = 1 ')
plt.xlabel('$ k, M^{-1} $')
plt.ylabel('$\delta$')
plt.plot(k,(k/dx*np.pi)**2,label='исходное уравнение')
plt.legend()
plt.title("Диффузия")

NameError: name 'gamma1' is not defined

<Figure size 1080x576 with 0 Axes>